In [2]:
import pandas as pd
import yfinance as yf
import quandl
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# Install necessary packages
%pip install yfinance
%pip install quandl
%pip install sklearn
%pip install -U scikit-learn scipy matplotlib

# Fetching the data
quandl.ApiConfig.api_key = "r-y_sbK2e3ZmXP9KZhFS"
unemployment_data = quandl.get("FRED/UNRATE")
interest_rate_data = quandl.get("FRED/DGS10")
inflation_data = quandl.get("FRED/CPIAUCNS")

# Download S&P 500 data
sp500 = yf.Ticker("^GSPC")
sp500 = sp500.history(period="max")

# Ensure the index is timezone-naive
sp500.index = sp500.index.tz_localize(None)

# Prepare S&P 500 data
sp500 = sp500.drop(columns=["Dividends", "Stock Splits"])
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.loc["1990-01-01":].copy()

# Resample economic data to daily frequency and fill missing values
unemployment_data = unemployment_data.resample('D').interpolate(method='linear').ffill().bfill()
interest_rate_data = interest_rate_data.resample('D').interpolate(method='linear').ffill().bfill()
inflation_data = inflation_data.resample('D').interpolate(method='linear').ffill().bfill()

# Ensure the indices of the economic data are timezone-naive
unemployment_data.index = unemployment_data.index.tz_localize(None)
interest_rate_data.index = interest_rate_data.index.tz_localize(None)
inflation_data.index = inflation_data.index.tz_localize(None)

# Merge economic data with S&P 500 data
sp500 = sp500.merge(unemployment_data, left_index=True, right_index=True, how='left')
sp500 = sp500.merge(interest_rate_data, left_index=True, right_index=True, how='left')
sp500 = sp500.merge(inflation_data, left_index=True, right_index=True, how='left')

# Rename columns for clarity
sp500.rename(columns={'Value_x': 'Unemployment_Rate', 'Value_y': 'Interest_Rate', 'Value': 'Inflation_Rate'}, inplace=True)

# Create new predictors based on rolling averages
horizons = [2, 5, 60, 250, 1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()

    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]

    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]

    new_predictors += [ratio_column, trend_column]

# Add economic indicators to predictors list
new_predictors += ['Unemployment_Rate', 'Interest_Rate', 'Inflation_Rate']

# Drop rows with missing values
sp500 = sp500.dropna()

# Define the model
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

# Update the predict function to handle probabilities
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:, 1]
    preds[preds >= .6] = 1
    preds[preds < .6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

# Backtest function remains the same
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)

# Function to predict a single day and show actual outcome
def predict_single_day(date, data, predictors, model):
    if date not in data.index:
        raise ValueError("Date not in data index")
    
    train = data[data.index < date]
    test = data[data.index == date]
    
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    actual = test["Target"].iloc[0]
    return preds[0], actual

# Train the model on the full dataset
model.fit(sp500[new_predictors], sp500["Target"])

# Example of predicting for a specific date
date_to_predict = "2018-05-01"
prediction, actual = predict_single_day(date_to_predict, sp500, new_predictors, model)
print(f"Prediction for {date_to_predict}: {'Up' if prediction == 1 else 'Down'}")
print(f"Actual for {date_to_predict}: {'Up' if actual == 1 else 'Down'}")

# Run backtest
predictions = backtest(sp500, model, new_predictors)
print(predictions["Predictions"].value_counts())
print(f"Overall accuracy: {precision_score(predictions["Target"], predictions["Predictions"])}")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Prediction for 2018-05-01: Up
Actual for 2018-05-01: Down
Predictions
0.0    3995
1.0     569
Name: count, dtype: int64
Overall accuracy: 0.5799648506151143
